# Rigid Registration

This note book contains the code/algorithm required to perform the rigid registration.
Note: Initially aretefacts are removed from both image using a blob detector. The blob detector is not very sophisticated and is based on the assumption that the biggest blob in the pictures provided is the tissue (both for the MALDI and the HandE image)

In [1]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
import SimpleITK as sitk
%matplotlib inline
from ipywidgets import widgets
from ipywidgets import fixed as fxd
import sys
import os
import tools


### Change the flags below to your preference

In [3]:
FOLDER_PATH = ''
FIXED_IMAGE_FILE_PATH = '' #Enter the MALDI optical image file path
MOVING_IMAGE_FILE_PATH = '' #Enter the HandE image file path
DOWNSAMPLE = True # Chnage this to false if you wish to attempt regisration without downsampling
FACTOR = 0.2
FEATURE_DETECTOR = 'AKAZE'
MIN_MATCH_COUNT = 10 # Choose the number minimum feature matches required


if not os.path.exists(FOLDER_PATH + 'RegisteredImages'):
    os.makedirs(FOLDER_PATH + 'RegisteredImages')
    FOLDER_PATHNEW = FOLDER_PATH + 'RegisteredImages/'
    print FOLDER_PATHNEW
else:
    FOLDER_PATHNEW = FOLDER_PATH

In [215]:
movingImage = cv2.imread(MOVING_IMAGE_FILE_PATH) # Here we read in the images, the file location given by above.
fixedImage = cv2.imread(FIXED_IMAGE_FILE_PATH )  

### Pre-processing code:
The following code is a set of methods to clean up the provided images. Note that these steps are specific to the HandE and MALDI images of this project. As both of these images are bi-modal images, a very simple and straightforward blob detection approach is used to isolate and extract the area of the tissue. 

(Please see the tools.py file for each of the function that is called here)


Note: The functions described below should be used in the given order: detectTissue()->extractTissu()->boundRectTissue()

(detectTissue()): First a threshold is applied to the image - that is all values of pixels above a certain threshold are sent to 255. We then apply morphological transformations (open and close) which will get rid of smaller blobs (noise) as well as holes in the bigger blobs. The output of this is then used as a mask to produce a binary image. This binary image should contain a mask around the whole of this tissue (the subject of the image that we are interested in) as well as other significantly big artifacts.

In [ ]:
mIBlobs = tools.detectTissue(img = movingImage, threshold = 200 , kernelSize = 41 , erosionIteration = 2) 
fIBlobs = tools.detectTissue(img = fixedImage, threshold = 200 , kernelSize = 41 , erosionIteration = 2)
plt.subplot(121), plt.imshow(mIBlobs)
plt.subplot(122), plt.imshow(fIBlobs)
plt.show()

(extractTissue()) We can then use the binary image from above to extract the tissue. We first compute the contours around the blobs, and then identify the one with the biggest contour area - this of course must be the tissue. This is then used to cut out just the tissue from the original image and place it on a white background (you can specify the background color if you wish)

In [ ]:
movingImageCleaned,mIContours,mIIndex =  tools.extractTissue (img = movingImage,blobs = mIBlobs, backgroundColour = (255,255,255))
fixedImageCleaned,fIContours,fIIndex =  tools.extractTissue (img = fixedImage,blobs = fIBlobs, backgroundColour = (255,255,255))

plt.subplot(121), plt.imshow(movingImageCleaned)
plt.subplot(122), plt.imshow(fixedImageCleaned)
plt.show()


(boundRectTissue()) Here we simply draw the bounding rectangle around the tissue, and extract only what is inside this area.

In [ ]:
movingBRectImage,x1,y1,w1,h1 = tools.boundRectTissue(img = movingImageCleaned, contours= mIContours, idx = mIIndex)
fixedBRectImage,x2,y2,w2,h2 = tools.boundRectTissue(img = fixedImageCleaned, contours= fIContours, idx = fIIndex)

plt.subplot(121), plt.imshow(movingBRectImage)
plt.subplot(122), plt.imshow(fixedBRectImage)
plt.show()

# Registration Code

Please see the Project Report for further details of the theory of feature detectors and feature matching. Breifly the code follows these steps:
- We do further process the images for feature detection by applying a blur/denoising. This is done by the preProc() and preProcLowRes() functions. In the case we opted to register a lower resolution version of the images we also down sample the images using the downSample() function.
- a feature detection algorithm is used to find features in both images. This is done first using the cv2.AKAZE_create() or cv2.ORB_create() to initiate the feature detector object. In principle you can use any other feature detector available in Open CV.
- Once this is done detectAndCompute() will find and return the features (also called key points) and the feature detectors. 
- We then find the matches by "brute force", that is we consider all posible matches between feature descriptors and match those who are the closest neighbours. We use cv2.BFMatcher() and knnMatch() to do this.
- The matches array will have a lot of false matches, which we atttempt to elminate using the ratio test. 
- We then estimate a partial affine transformation (technically should be called a similarity transformation), which uses the RANSAC iterative method.



In [219]:
mImageGray = cv2.cvtColor(movingBRectImage, cv2.COLOR_BGR2GRAY)
fImageGray = cv2.cvtColor(fixedBRectImage, cv2.COLOR_BGR2GRAY)
if DOWNSAMPLE == True :
    
    fImageDS =  tools.downSample(fixedImage,FACTOR)
    mImageDS =  tools.downSample(movingImage,FACTOR)    
    
    
    fImageGrayDS = tools.downSample(fImageGray,FACTOR) #downsampled fixed image bounded rectangle
    mImageGrayDS = tools.downSample(mImageGray,FACTOR) #downsampled img1
    
    mImageGrayHisto = tools.preProc(mImageGrayDS,11,50)
    fImageGrayHisto = tools.preProc(fImageGrayDS,11,50)
    
    
    mImageGrayHisto = tools.regImageReposition( mImageGrayHisto,  mImageDS, (int(x1*FACTOR),int(y1*FACTOR),int(w1*FACTOR),int(h1*FACTOR)) , True)
    fImageGrayHisto = tools.regImageReposition( fImageGrayHisto,  fImageDS, (int(x2*FACTOR),int(y2*FACTOR),int(w2*FACTOR),int(h2*FACTOR)), True)
    
else:
    mImageGrayHisto = tools.preProc(mImageGray,73,300)
    fImageGrayHisto = tools.preProc(fImageGray,73,300)
    
    mImageGrayHisto = tools.regImageReposition( mImageGrayHisto,  movingImage, (x1,y1,w1,h1) ,  True)
    fImageGrayHisto = tools.regImageReposition( fImageGrayHisto,  fixedImage, (x2,y2,w2,h2) , True)
    
RERUN = True     # This is a varaible that will ensure that the image varaibles are re-initated in case there has been changes in the preprocess.

Pre-proc complete. Completed histo equalization and blurring
Pre-proc complete. Completed histo equalization and blurring


In [220]:
featureDetector = None
# Initiate feature detector:
if FEATURE_DETECTOR == 'AKAZE':
    featureDetector = cv2.AKAZE_create()
elif FEATURE_DETECTOR == 'ORB':
    featureDetector = cv2.ORB_create()

# find the keypoints and descriptors with feature detector
kp1, des1 = featureDetector.detectAndCompute(mImageGrayHisto,None) # finding the kp and des of the HandE image
kp2, des2 = featureDetector.detectAndCompute(fImageGrayHisto,None) # finding the kp and des of the MALDI image
print 'Completed  detection...'

Completed  detection...


In [221]:
if FEATURE_DETECTOR == 'AKAZE' or FEATURE_DETECTOR == 'ORB':
    bf = cv2.BFMatcher(cv2.NORM_HAMMING) # Here we initiate the Brute force method object - this will essentially iterate through all possible matches hence brute force
    matches = bf.knnMatch(des1,des2,k=2) # Finds the K nearest neighbours using the fecture descriptors
print 'Completed Key Point Matching...'

Completed Key Point Matching...


In [232]:
ratioTestArray = [] # This array will contain all the refined matches, as the bf matcher output will contain some flase matches
for m,n in matches:
    if m.distance < 0.7*n.distance: # Here we simply use the ratio test as suggested by (D.Lowe, 2004)
        ratioTestArray.append(m) # If the ratio between the 

if len(ratioTestArray)>MIN_MATCH_COUNT:
    print "Enough matches found"
    print "Matches:",len(ratioTestArray)
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in ratioTestArray ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in ratioTestArray ]).reshape(-1,1,2)
    
    M1, mask= cv2.estimateAffinePartial2D(src_pts,dst_pts,method=cv2.RANSAC,ransacReprojThreshold= 5.0, maxIters = 2000)
    matchesMask1 = mask.ravel().tolist()

    
    if DOWNSAMPLE == True and M1 != None:
        w,h,_ = fixedImage.shape
        
        wDS = int(w * FACTOR)
        hDS = int(h * FACTOR)
        

        regImageDS = cv2.warpAffine(mImageDS, M1, (hDS,wDS),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))
        
        print 'Down Sampled transform matrix:'
        print M1
        
        M2 = M1
        M2[:,2] = M2[:,2]*(1/FACTOR)
        regImage = cv2.warpAffine(movingImage, M2, (h,w),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))

        
        print 'Final transform matrix:'
        print M2
        
    elif DOWNSAMPLE == False and M1 != None:
        w,h = fImageGrayHisto.shape
        regImage = cv2.warpAffine(movingImage, M1, (h,w),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))
        
        print 'No downsample estimated.Final transform matrix:'
        print M1
        
    elif M1 == None:
        print 'No transform found, check matching is correct'
        
else:
    print "Not enough matches are found - %d/%d" % (len(ratioTestArray),MIN_MATCH_COUNT)
    matchesMask = None

Enough matches found
Matches: 25
Down Sampled transform matrix:
[[  7.77292492e-01   2.49900938e-02   2.03018965e+02]
 [ -2.49900938e-02   7.77292492e-01   1.82738482e+02]]
Final transform matrix:
[[  7.77292492e-01   2.49900938e-02   1.01509483e+03]
 [ -2.49900938e-02   7.77292492e-01   9.13692410e+02]]


C:\Users\dt716997\Anaconda\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  app.launch_new_instance()


### Output : 
This will show the overlay of the two images, can be used to evaluate the registration visually.

In [ ]:
if RERUN == True: # We preprocess the images so that it shows better when presented as an overaly.
    #fImageGrayHisto = cv2.cvtColor(fImageGrayHisto, cv2.COLOR_GRAY2BGR)
    if DOWNSAMPLE == True :
        fImageGrayHistoUS = tools.preProc(fImageGray,73,75)
        fImageGrayHistoUS = tools.regImageReposition( fImageGrayHistoUS,  fixedImage, (x2,y2,w2,h2) , True)
        fImageGrayHistoUS = cv2.cvtColor(fImageGrayHistoUS, cv2.COLOR_GRAY2BGR)
RERUN = False



if DOWNSAMPLE == True :
    
    print fImageGrayHisto.shape
    print regImageDS.shape
    
    dDestDS = cv2.addWeighted(regImageDS,0.5,fImageGrayHisto,0.5,gamma = 0)
    
    
    
    dDestUS = cv2.addWeighted(regImage,0.5,fImageGrayHistoUS,0.5,gamma = 0)
    
    
    plt.subplot(121), plt.imshow(dDestDS)
    plt.subplot(122), plt.imshow(dDestUS)
    
    
else:
    dDest2 = cv2.addWeighted(regImage,0.5,fImageGrayHisto,0.5,gamma = 0)
    plt.imshow(dDest2)

In [234]:
if DOWNSAMPLE == True :
    cv2.imwrite(FOLDER_PATHNEW + 'overlayLowRes.jpg',dDestDS)
    cv2.imwrite(FOLDER_PATHNEW + 'overlayHighRes.png',dDestUS)
    cv2.imwrite(FOLDER_PATHNEW + 'regImageDS.jpg',regImageDS)
    cv2.imwrite(FOLDER_PATHNEW + 'regImageUS.jpg',regImage)
else:
    cv2.imwrite(FOLDER_PATHNEW + 'rigRegImage.jpg',regImage)
    cv2.imwrite(FOLDER_PATHNEW + 'rigRegoverlay.jpg',dDest2)

In [ ]:
def testing(fixedImage,HandE):
    plt.figure(figsize=(20,10))
    img3__ = plt.imshow(fImageGrayHisto, alpha=fixedImage)
    img3__ = plt.imshow(regImageDS, alpha=HandE)
    plt.show()

widgets.interact(testing,fixedImage = (0,1,0.2) ,HandE = (0,1,0.2))

You can further analyse:
    - the images used to detect features (histogram equalized and blur filter)
    - the macthes prior and post RANSAC

In [235]:
if DOWNSAMPLE == True :
    
    
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matchesMask1, # draw only inliers
                       flags = 2)

    img3 = cv2.drawMatches(mImageDS,kp1,fImageDS,kp2,ratioTestArray,None,**draw_params)


    draw_params1 = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = None, # draw only inliers
                       flags = 2)

    img4 = cv2.drawMatches(mImageDS,kp1,fImageDS,kp2,ratioTestArray,None,**draw_params1)

    
else:    

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matchesMask1, # draw only inliers
                       flags = 2)

    img3 = cv2.drawMatches(movingImage,kp1,fixedImage,kp2,ratioTestArray,None,**draw_params)


    draw_params1 = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = None, # draw only inliers
                       flags = 2)

    img4 = cv2.drawMatches(movingImage,kp1,fixedImage,kp2,ratioTestArray,None,**draw_params1)

In [236]:
cv2.imwrite(FOLDER_PATHNEW + 'Matches_RANSACMask.jpg',img3)
cv2.imwrite(FOLDER_PATHNEW + 'Matches_NoMask.jpg',img4)
cv2.imwrite(FOLDER_PATHNEW + 'histoimagesMI.jpg',mImageGrayHisto)
cv2.imwrite(FOLDER_PATHNEW + 'histoimagesFI.jpg',fImageGrayHisto)

True

# IMPORTANT !
Please run the cell below to generate the histo-equlized image of the registered image. This is the image you will be using as the moving image when doing the deformable registration.

In [239]:
if DOWNSAMPLE == True :
    mImageGrayHisto = tools.preProc(mImageGray,73,75)
    mImageGrayHisto = tools.regImageReposition( mImageGrayHisto,  movingImage, (x1,y1,w1,h1) ,  True)
    regImageHisto = cv2.warpAffine(mImageGrayHisto, M2, (h,w),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))
else:
    regImageHisto = cv2.warpAffine(mImageGrayHisto, M2, (h,w),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))

cv2.imwrite(FOLDER_PATHNEW + 'regImageHisto.jpg',regImageHisto)
cv2.imwrite(FOLDER_PATHNEW + 'mImageHistoUS.jpg',fImageGrayHistoUS)

Pre-proc complete. Completed histo equalization and blurring


True